In [1]:
import requests
import sqlite3
from datetime import date

def obtener_precio_bitcoin():
    try:
        response = requests.get("https://api.coindesk.com/v1/bpi/currentprice.json")
        response.raise_for_status()
        return response.json()["bpi"]
    except requests.RequestException as e:
        print("Error al obtener el precio de Bitcoin:", e)
        return None

def obtener_tipo_cambio_sunat(fecha):
    try:
        url = f"https://api.apis.net.pe/v1/tipo-cambio-sunat?fecha={fecha}"
        response = requests.get(url)
        response.raise_for_status()
        return response.json()['compra']
    except requests.RequestException as e:
        print("Error al obtener el tipo de cambio:", e)
        return None

def guardar_precio_bitcoin_en_db():
    precios_bitcoin = obtener_precio_bitcoin()
    if precios_bitcoin is None:
        return
    
    fecha_actual = date.today().strftime('%Y-%m-%d')
    tipo_cambio_usd = obtener_tipo_cambio_sunat(fecha_actual)
    if tipo_cambio_usd is None:
        return
    
    precio_pen = precios_bitcoin['USD']['rate_float'] * tipo_cambio_usd
    
    conn = sqlite3.connect('base.db')
    cursor = conn.cursor()
    
    cursor.execute('''CREATE TABLE IF NOT EXISTS bitcoin (
                        FECHA TEXT PRIMARY KEY,
                        PRECIO_USD REAL,
                        PRECIO_GBP REAL,
                        PRECIO_EUR REAL,
                        PRECIO_PEN REAL
                    )''')
    
    cursor.execute("INSERT INTO bitcoin (FECHA, PRECIO_USD, PRECIO_GBP, PRECIO_EUR, PRECIO_PEN) VALUES (?, ?, ?, ?, ?)",
                   (fecha_actual, precios_bitcoin['USD']['rate_float'], precios_bitcoin['GBP']['rate_float'], 
                    precios_bitcoin['EUR']['rate_float'], precio_pen))
    
    conn.commit()
    conn.close()
    
    print("Datos de Bitcoin guardados en la tabla 'bitcoin' de la base de datos 'base.db'")

# Llamar a la función para guardar los datos
guardar_precio_bitcoin_en_db()


Datos de Bitcoin guardados en la tabla 'bitcoin' de la base de datos 'base.db'


OBSERVANDO LA TABLA BITCOIN

In [2]:
import sqlite3

with sqlite3.connect('base.db') as conexion:
    cursor = conexion.cursor()

    # Recuperamos los registros de la tabla de usuarios
    cursor.execute("SELECT * FROM bitcoin")

    # Recorremos todos los registros con fetchall
    # y los volcamos en una lista de usuarios
    usuarios = cursor.fetchall()
    pass

# Ahora podemos recorrer todos los usuarios
for usuario in usuarios:
    print(usuario)

('2024-02-14', 49558.9776, 39318.4574, 46254.5336, 192140.15615519998)


REALIZANDO LA CONSULTA

In [3]:
import sqlite3

def calcular_precio_compra(bitcoins, moneda):

    conn = sqlite3.connect('base.db')
    cursor = conn.cursor()

    # Consultar el precio actual del bitcoin en la moneda deseada
    cursor.execute(f"SELECT PRECIO_{moneda.upper()} FROM bitcoin ORDER BY FECHA DESC LIMIT 1")
    precio_bitcoin = cursor.fetchone()[0]  # Obtener el precio de bitcoin en la moneda deseada

    # Calcular el precio de compra de los bitcoins en la moneda deseada
    precio_compra = bitcoins * precio_bitcoin

    # Cerrar la conexión
    conn.close()

    return precio_compra

# Calcular el precio de compra de 10 bitcoins en PEN
precio_compra_pen = calcular_precio_compra(10, 'PEN')
print(f"El precio de compra de 10 bitcoins en PEN es: {precio_compra_pen:.2f} PEN")

# Calcular el precio de compra de 10 bitcoins en EUR
precio_compra_eur = calcular_precio_compra(10, 'EUR')
print(f"El precio de compra de 10 bitcoins en EUR es: {precio_compra_eur:.2f} EUR")


El precio de compra de 10 bitcoins en PEN es: 1921401.56 PEN
El precio de compra de 10 bitcoins en EUR es: 462545.34 EUR
